<a href="https://colab.research.google.com/github/ailab-nda/ML/blob/main/Trying_Langchain_RAG_with_Elyza_instruct_7b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Elyza 7bを用いたRAGを試してみた。

今回はLLMには[Elyza 7B Instruct](https://huggingface.co/elyza/ELYZA-japanese-Llama-2-7b-fast-instruct)を用い、Langchainを使ったRAG (Retrieval Augmented Generation) を試してみました。

RAGを用いることで質問に対して関連性の高い文章を抽出し、より適切な答えを導き出せることを期待します。


## 必要なライブラリをインストール


In [ ]:
# To solve for an error encountered: `NotImplementedError: A UTF-8 locale is required. Got ANSI_X3.4-1968`
import locale
locale.getpreferredencoding = lambda: "UTF-8"

# 必要なライブラリをインストール
!pip install transformers langchain accelerate bitsandbytes pypdf tiktoken sentence_transformers faiss-gpu trafilatura --quiet

In [ ]:
# テキストが見やすいようにwrapしておく
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

## コードを実行

必要なライブラリをロードします。


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms.huggingface_pipeline import HuggingFacePipeline
from langchain import PromptTemplate

## データソースを準備

今回はウィキペディア上にある「ONE PIECE」のページをデータソースとして、それに関連する質問をしていきたいと思います。

* https://ja.m.wikipedia.org/wiki/ONE_PIECE

今回はウェブページのテキストだけを抽出してくれる `trafilatura` というライブラリを用いましたが、Langchain内にもこれ用の `BSHTMLLoader` というのがあるようです。まだ試せていません。

In [ ]:
# BSHTMLLoaderを使う場合のサンプルコード
# https://python.langchain.com/docs/modules/data_connection/document_loaders/html から引用。
# from langchain.document_loaders import BSHTMLLoader

# loader = BSHTMLLoader("example_data/fake-content.html")
# data = loader.load()
# data

In [ ]:
from trafilatura import fetch_url, extract

url = "https://ja.m.wikipedia.org/wiki/ONE_PIECE"
filename = 'textfile.txt'

document = fetch_url(url)
text = extract(document)
print(text[:1000])

with open(filename, 'w', encoding='utf-8') as f:
    f.write(text)

抽出したテキストをテキストファイルに保存出来ました。このテキストファイルをLangchainのTextSplitterを使って小口のチャンクに切っていきます。

こうして生成したチャンクからembeddingを生成し、質問のembeddingに一番近いトップｋのチャンクを抽出。そのテキストをプロンプト内に突っ込み、質問と同時にLLMに投げて回答を得る。といった流れとなります。

私は少なくともそういう理解です。

In [ ]:
loader = TextLoader(filename, encoding='utf-8')
documents = loader.load()

text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator = "\n",
    chunk_size=300,
    chunk_overlap=20,
)
texts = text_splitter.split_documents(documents)
print(len(texts))

どんな構造をしているのか知るために、何個か見てみましょう。

In [ ]:
texts[30:33]

## Embeddingの生成とFAISSを使ったベクトルDBの用意

小口のチャンクに切ったテキストを、Embeddingモデルを使ってembeddingに変換していきます。テキストの類似性をもとに検索をできるようにするためです。

Embeddingの生成には `intfloat/multilingual-e5-large` を使います。

ベクトルDBには `FAISS` のライブラリを用います。（今回はGPUのある環境で走らせてみているため、 faiss-gpu をロードしています。）

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large")
db = FAISS.from_documents(texts, embeddings)

# 一番類似するチャンクをいくつロードするかを変数kに設定出来ます。
retriever = db.as_retriever(search_kwargs={"k": 5})

今回の環境ではembeddingを用意するのに25秒ほどかかりました。

## モデルの用意

今回はElyza-7b-instructを用います。

今回はColabのT4でも問題なく実行できるよう、BitsandBytesで4bitに量子化したものをロードします。

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "elyza/ELYZA-japanese-Llama-2-7b-instruct"

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    quantization_config=quantization_config,
).eval()

次に、Elyza-7b-instruct用のプロンプトテンプレートを用意します。

In [ ]:
B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
DEFAULT_SYSTEM_PROMPT = "参考情報を元に、ユーザーからの質問にできるだけ正確に答えてください。"
text = "{context}\n質問: {question}"
template = "{bos_token}{b_inst} {system}{prompt} {e_inst} ".format(
    bos_token=tokenizer.bos_token,
    b_inst=B_INST,
    system=f"{B_SYS}{DEFAULT_SYSTEM_PROMPT}{E_SYS}",
    prompt=text,
    e_inst=E_INST,
)

## LLMとChainの指定

In [ ]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
)
PROMPT = PromptTemplate(
    template=template,
    input_variables=["context", "question"],
    template_format="f-string"
)

chain_type_kwargs = {"prompt": PROMPT}

qa = RetrievalQA.from_chain_type(
    llm=HuggingFacePipeline(
        pipeline=pipe,
        model_kwargs=dict(
            temperature=0.0,
            do_sample=True,
            max_length=512,
            repetition_penalty=1.1
        )
    ),
    retriever=retriever,
    chain_type="stuff",
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs,
    verbose=True,
)

## お試し

要約質問ができる状態が整いました。
最初にRAGを使わずにLLMに質問をし、その後にRAGを使って生成してみて差分を比較してみましょう。

In [ ]:
inputs = template.format(context='', question='ニコ・ロビンの職業は何ですか？')
inputs = tokenizer(inputs, return_tensors='pt').to(model.device)

with torch.no_grad():
    output_ids = model.generate(
        **inputs,
        max_new_tokens=512,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id,
    )
output = tokenizer.decode(output_ids.tolist()[0], skip_special_tokens=True)
output

In [ ]:
result = qa("ニコ・ロビンの職業は何ですか？")
print('回答:', result['result'])
print('='*10)
print('ソース:', result['source_documents'])

In [ ]:
inputs = template.format(context='', question='エネルは何者ですか？')
inputs = tokenizer(inputs, return_tensors='pt').to(model.device)

with torch.no_grad():
    output_ids = model.generate(
        **inputs,
        max_new_tokens=512,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id,
    )
output = tokenizer.decode(output_ids.tolist()[0], skip_special_tokens=True)
output

In [ ]:
result = qa("エネルは何者ですか？")
print('回答:', result['result'])
print('='*10)
print('ソース:', result['source_documents'])

In [ ]:
inputs = template.format(context='', question='チョッパーの特殊能力は何ですか？')
inputs = tokenizer(inputs, return_tensors='pt').to(model.device)

with torch.no_grad():
    output_ids = model.generate(
        **inputs,
        max_new_tokens=512,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id,
    )
output = tokenizer.decode(output_ids.tolist()[0], skip_special_tokens=True)
output

In [ ]:
result = qa("チョッパーの特殊能力は何ですか？")
print('回答:', result['result'])
print('='*10)
print('ソース:', result['source_documents'])

In [ ]:
inputs = template.format(context='', question="「ONE PIECE」とは作品の中で何を指していますか？")
inputs = tokenizer(inputs, return_tensors='pt').to(model.device)

with torch.no_grad():
    output_ids = model.generate(
        **inputs,
        max_new_tokens=512,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id,
    )
output = tokenizer.decode(output_ids.tolist()[0], skip_special_tokens=True)
output

In [ ]:
result = qa("「ONE PIECE」とは作品の中で何を指していますか？")
print('回答:', result['result'])
print('='*10)
print('ソース:', result['source_documents'])

## 結論

RAGにより回答の質が全体的にかなり上がったことが確認できました。

余談：最後の質問に対するGPT-4のRAGなしでの回答は下記でした。流石ですね:

`
「ONE PIECE」は、日本の漫画家尾田栄一郎（Eiichiro Oda）によって作られた漫画およびアニメ作品であり、その中で「One Piece」とは、伝説的な海賊ゴール・D・ロジャーが残したとされる、世界最大の財宝を指します。この財宝は、最も危険で未知な海域である「偉大なる航路（Grand Line）」の最後にある「ラフテル」という島に隠されているとされています。
`

## 参考

こちらのノートブックを作成するにあたり、下記を参考にさせていただいております。

* [alfredplpl/RetrievalQA.py](https://gist.github.com/alfredplpl/57a6338bce8a00de9c9d95bbf1a6d06d)
* [Langchain Docs](https://python.langchain.com/docs/get_started/introduction)
* [Wikipedia「ONE_PIECE」](https://ja.m.wikipedia.org/wiki/ONE_PIECE)

